In [1]:
import sys, os
sys.path.append('./venv/lib/site-packages')
sys.path.append('./venv/poppler/site-packages')

In [4]:
# https://www.geeksforgeeks.org/python-reading-contents-of-pdf-using-ocr-optical-character-recognition/
# Import libraries 
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path, convert_from_bytes

from base64 import b64decode
import codecs


In [5]:
base64String="..."

In [6]:
def base64toPdf(base64String):
    bytes = b64decode(base64String, validate=True)

    if bytes[0:4] != b'%PDF':
      raise ValueError('Missing the PDF file signature')

    with open("file.pdf", "wb") as fh:
        fh.write(bytes)
    return('file.pdf')


In [7]:
base64toPdf(base64String)

'file.pdf'

In [8]:
def PdftoImage(PdfFile):
    pages = convert_from_path('file.pdf', poppler_path=r'.\venv\poppler\bin')
    for page in pages:
        page.save('out.jpg', 'JPEG')

In [24]:
PdftoImage('file.pdf')

In [10]:
bytesStr = b64decode(base64String, validate=True)

if bytesStr[0:4] != b'%PDF':
    raise ValueError('Missing the PDF file signature')

pages = convert_from_bytes(bytesStr, fmt='jpeg', poppler_path=r'.\venv\poppler\bin')

pytesseract.pytesseract.tesseract_cmd = './venv/Tesseract-OCR/tesseract.exe'

print(pytesseract.image_to_string(pages[0], lang='spa'))

e m a ” u

 

Scanned by TapScanner

